In [2]:
import sys
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# LangChain 관련 라이브러리
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
# RAGAS 관련 라이브러리
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

In [5]:
import pandas as pd

dataset_df = pd.read_json("qa_dataset.jsonl", lines=True)
display(dataset_df)

,question,reference,answer
0,인사 변화의 필요성을 분석하는 책임자는 누구인가요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 필요성 분석의 책임자는 인사팀입니다.
1,승진 및 이동 결정 시 어떤 기준을 수립해야 하나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,승진 및 이동 결정 시 후보자 선정 및 평가 기준을 CTO와 인사팀이 함께 수립해야...
2,인사 변화 공지는 언제 이루어지나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 공지는 2025-09-30에 인사팀에 의해 이루어집니다.
3,인사 변화 후 어떤 점을 모니터링해야 하나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 후에는 조직 및 팀 성과에 미치는 영향을 지속적으로 모니터링해야 합니다.
4,조직 구조 개편안의 목적은 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,조직 구조 개편은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 위한 전략...
5,조직 구조 개편안의 적용 범위는 어떻게 되나요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,"적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)입니다."
6,조직 구조 개편안의 실행 계획에서 1단계는 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,"1단계는 현 조직 구조 분석 및 문제점 도출로, 기간은 2주이며 책임자는 인사팀입니다."
7,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,직원 저항에 대한 대응 방안은 충분한 커뮤니케이션 및 교육 제공입니다.
8,핵심 인재 유출 리스크를 평가하기 위한 첫 번째 단계는 무엇인가요?,[# 인사·조직 기밀 | 핵심 인재 유출 리스크 분석 및 대응 전략\n\n작성일: ...,"첫 번째 단계는 리스크 진단이며, 목표는 현재 인재 유출 리스크를 평가하는 것입니다."
9,인재 유출을 예방하기 위한 조치에는 어떤 것들이 있나요?,[# 인사·조직 기밀 | 핵심 인재 유출 리스크 분석 및 대응 전략\n\n작성일: ...,"인재 유출을 예방하기 위한 조치로는 보상 체계 재검토 및 개선, 경력 개발 및 교육..."


## 02.RAG 체인 구성

검색기와 생성 모델을 결합한 RAG 체인을 구성한다.

In [6]:
from dotenv import load_dotenv

load_dotenv()

from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils_qwen3_topk").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils_qwen3_topk.rag import chat

response = chat({"history": [
    {"role": "user", "content": "코드노바의 API 서버 기술스택알려줘"}], "tone":"formal", "permission":"cto", "k":7})

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:utils_qwen3_topk.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils_qwen3_topk.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')

In [7]:
print(response['response'])

코드노바의 API 서버 기술스택은 Node.js와 Express.js로 구성되어 있습니다.


In [8]:
print(response['result'])
type(response['result'])

['검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->\n# 서비스 아키텍처 문서\n분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n## 1. 개요\n코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. API 서버\n- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.\n- **기술 스택**: Node.js, Express.js\n- **검증 포인트**:\n  - API 엔드포인트가 올바르게 작동하는지 확인\n  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]\n<!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->\n# 프론트엔드 아키텍처 문서\n분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n---\n\n## 1. 개요\n코드노바의 프론트엔드 아키텍처는 사용자 경험을 극대화하고, 유지보수성을 높이며, 확장성을 고려하여 설계되었습니다. 본 문서는 프론트엔드 개발자가 이해하고 실행할 수 있도록 아키텍처의 구성 요소와 원칙을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. 기술 스택\n- **프레임워크**: React, Vue.js 또는 Angular 중 하나를 선택하여 사용\n- **상태 관리**: Redux, Vuex 또는 Context API를 사용하여 애플리케이션 상태 관리\n- **스타일링**: CSS Modules, Styled-components 또는 SCSS 사용\n- **빌드 도구**: Webpack, Babel 등 

list

## 03.RAGAS기반 평가

RAGAS는 RAG(Retrieval-Augmented Generation) 시스템을 평가하는 자동화된 참조 없는(reference-free) 평가 프레임워크입니다. 즉, 사람이 만든 정답 데이터(ground truth)가 없이도 RAG 시스템의 검색과 생성 단계를 자동으로 평가할 수 있도록 설계된 도구입니다. RAGAS는 LLM을 활용해 평가를 수행하며, 검색된 컨텍스트와 생성된 답변의 충실도, 관련성, 정밀도, 재현율 등 다양한 측면을 측정합니다.

### RAGAS 주요 평가 지표
- **Faithfulness(충실도)**: 생성된 답변이 주어진 컨텍스트 정보에 얼마나 충실한지를 평가합니다. 답변 내용이 컨텍스트에서 실제로 뒷받침되는지 보는 지표입니다.
- **Answer Relevancy(답변 관련성)**: 답변이 원 질문과 얼마나 관련성이 높은지를 측정합니다.
- **Context Precision(컨텍스트 정밀도)**: 검색된 컨텍스트 문서가 질문에 적절한 정보인지, 관련된 문서가 상위에 있는지를 평가합니다.
- **Context Recall(컨텍스트 재현율)**: 답변을 생성하는 데 필요한 컨텍스트를 얼마나 잘 검색했는지 평가합니다.


In [9]:
# 평가용 데이터 로드
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_json("qa_dataset.jsonl", lines=True)
print(dataset_df.shape)

eval_dataset = dataset_df[['question', 'reference', 'answer']]
display(eval_dataset)

(55, 3)


,question,reference,answer
0,인사 변화의 필요성을 분석하는 책임자는 누구인가요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 필요성 분석의 책임자는 인사팀입니다.
1,승진 및 이동 결정 시 어떤 기준을 수립해야 하나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,승진 및 이동 결정 시 후보자 선정 및 평가 기준을 CTO와 인사팀이 함께 수립해야...
2,인사 변화 공지는 언제 이루어지나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 공지는 2025-09-30에 인사팀에 의해 이루어집니다.
3,인사 변화 후 어떤 점을 모니터링해야 하나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 후에는 조직 및 팀 성과에 미치는 영향을 지속적으로 모니터링해야 합니다.
4,조직 구조 개편안의 목적은 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,조직 구조 개편은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 위한 전략...
5,조직 구조 개편안의 적용 범위는 어떻게 되나요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,"적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)입니다."
6,조직 구조 개편안의 실행 계획에서 1단계는 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,"1단계는 현 조직 구조 분석 및 문제점 도출로, 기간은 2주이며 책임자는 인사팀입니다."
7,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,직원 저항에 대한 대응 방안은 충분한 커뮤니케이션 및 교육 제공입니다.
8,핵심 인재 유출 리스크를 평가하기 위한 첫 번째 단계는 무엇인가요?,[# 인사·조직 기밀 | 핵심 인재 유출 리스크 분석 및 대응 전략\n\n작성일: ...,"첫 번째 단계는 리스크 진단이며, 목표는 현재 인재 유출 리스크를 평가하는 것입니다."
9,인재 유출을 예방하기 위한 조치에는 어떤 것들이 있나요?,[# 인사·조직 기밀 | 핵심 인재 유출 리스크 분석 및 대응 전략\n\n작성일: ...,"인재 유출을 예방하기 위한 조치로는 보상 체계 재검토 및 개선, 경력 개발 및 교육..."


In [10]:
# 평가용 데이터셋 생성
evaluated_dataset = []

# 각 행에 대해 RAG 체인을 호출하여 결과를 저장
for row in eval_dataset.itertuples():
    query = row.question # 사용자 입력
    response = chat({"history": [
    {"role": "user", "content": query}], "tone":"formal", "permission":"cto", "k": 7})
    retrieved_contexts = response['result']  # 실제 검색된 문서

    reference = row.answer  # 정답


    evaluated_dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": retrieved_contexts,
            "response": response['response'],
            "reference": reference,
        }
    )

# RAGAS 평가 데이터셋 생성
ragas_evaluated_dataset = EvaluationDataset.from_list(evaluated_dataset)

# 데이터 저장
ragas_evaluated_dataset.to_pandas().to_csv('ragas_evaluated_dataset.csv', index=False)

INFO:utils_qwen3_topk.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils_qwen3_topk.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object with function name and

In [11]:
display(ragas_evaluated_dataset.to_pandas())

,user_input,retrieved_contexts,response,reference
0,인사 변화의 필요성을 분석하는 책임자는 누구인가요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관...,인사 변화의 필요성을 분석하는 책임자는 인사팀입니다.,인사 변화 필요성 분석의 책임자는 인사팀입니다.
1,승진 및 이동 결정 시 어떤 기준을 수립해야 하나요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관...,승진 및 이동 결정 시 다음과 같은 기준을 수립하는 것이 좋습니다.\n\n1. 성과...,승진 및 이동 결정 시 후보자 선정 및 평가 기준을 CTO와 인사팀이 함께 수립해야...
2,인사 변화 공지는 언제 이루어지나요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n| 단계 | 설명 |...,인사 변화 공지는 2025년 9월 30일에 이루어집니다.,인사 변화 공지는 2025-09-30에 인사팀에 의해 이루어집니다.
3,인사 변화 후 어떤 점을 모니터링해야 하나요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n| 단계 | 설명 |...,인사 변화 후 모니터링해야 할 주요 사항은 다음과 같습니다.\n\n1. 인사 변화가...,인사 변화 후에는 조직 및 팀 성과에 미치는 영향을 지속적으로 모니터링해야 합니다.
4,조직 구조 개편안의 목적은 무엇인가요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일:...,조직 구조 개편안의 목적은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 ...,조직 구조 개편은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 위한 전략...
5,조직 구조 개편안의 적용 범위는 어떻게 되나요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일:...,"조직 구조 개편안의 적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)이며, 대상...","적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)입니다."
6,조직 구조 개편안의 실행 계획에서 1단계는 무엇인가요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n\n| 단계 | 내용...,"조직 구조 개편안의 실행 계획 1단계는 ""현 조직 구조 분석 및 문제점 도출""입니다...","1단계는 현 조직 구조 분석 및 문제점 도출로, 기간은 2주이며 책임자는 인사팀입니다."
7,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 무엇인가요?,[검색 결과:\n-----\n## 4. 리스크 및 대응 방안\n\n| 리스크 | 설...,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 다음과 같습니다.\n\n- 충분...,직원 저항에 대한 대응 방안은 충분한 커뮤니케이션 및 교육 제공입니다.
8,핵심 인재 유출 리스크를 평가하기 위한 첫 번째 단계는 무엇인가요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n\n### 단계 1:...,"핵심 인재 유출 리스크 평가의 첫 번째 단계는 ""리스크 진단""입니다. 이 단계에서는...","첫 번째 단계는 리스크 진단이며, 목표는 현재 인재 유출 리스크를 평가하는 것입니다."
9,인재 유출을 예방하기 위한 조치에는 어떤 것들이 있나요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n\n### 단계 1:...,인재 유출을 예방하기 위한 조치는 다음과 같습니다.\n\n1. 보상 체계 재검토 및...,"인재 유출을 예방하기 위한 조치로는 보상 체계 재검토 및 개선, 경력 개발 및 교육..."


### RAGAS 평가 실행

**ragas.metrics 주요 클래스 설명**

- **LLMContextRecall**  
  
  주어진 답변이 원문(문맥)에서 제시된 정보를 얼마나 잘 회상(recall)했는지 평가하는 클래스이다.
  
  예를 들어, 원문에 중요한 사실 10개가 있는데 답변이 그중 몇 개를 잘 포함했는지 측정한다.
  즉, 잊지 않고 잘 회상했는지 보는 지표다.

- **Faithfulness**  
  답변이 원본(문맥) 정보에 기반하여 얼마나 충실하고 일관되게 생성되었는지 평가하는 클래스이다.

  답변 내 내용이 문맥과 어긋나지 않고, 문맥에서 유추 가능한 사실들로만 이루어졌는지를 본다.

  낮은 faithfulness 값은 모델이 문맥에 없는 정보를 만들어내거나 왜곡하여 답변함을 의미할 수 있어, 환각 발생 정도를 간접적으로 보여준다.

- **FactualCorrectness**  
  답변의 내용이 실제 사실과 얼마나 정확하게 일치하는지를 평가하는 클래스이다.

  문맥뿐 아니라 절대적인 사실관계(정확성)와 맞는지 따진다.

  즉, 문맥이 아니라 객관적인 사실에 맞는지를 측정하는 지표다.

In [12]:
# LLM 래퍼 생성
evaluator_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
evaluator_llm = LangchainLLMWrapper(evaluator_llm)

# 평가 메트릭 정의
metrics = [
    LLMContextRecall(),    # 검색된 컨텍스트의 회수율
    Faithfulness(),        # 생성된 답변의 충실도
    FactualCorrectness()   # 사실적 정확성
]

In [13]:
# 평가 실행
result = evaluate(
    dataset=ragas_evaluated_dataset,  # 평가 데이터셋
    metrics=metrics,             # 평가 메트릭
    llm=evaluator_llm,          # LLM 래퍼
)

print(result)

Evaluating:   0%|          | 0/165 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

{'context_recall': 0.9333, 'faithfulness': 0.8736, 'factual_correctness(mode=f1)': 0.3802}


In [14]:
# 결과를 DataFrame으로 변환
result_df = result.to_pandas()

# 결과 저장
result_df.to_csv('ragas_evaluation_result2.csv', index=False)

display(result_df)

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,인사 변화의 필요성을 분석하는 책임자는 누구인가요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관...,인사 변화의 필요성을 분석하는 책임자는 인사팀입니다.,인사 변화 필요성 분석의 책임자는 인사팀입니다.,1.000000,1.000000,1.00
1,승진 및 이동 결정 시 어떤 기준을 수립해야 하나요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관...,승진 및 이동 결정 시 다음과 같은 기준을 수립하는 것이 좋습니다.\n\n1. 성과...,승진 및 이동 결정 시 후보자 선정 및 평가 기준을 CTO와 인사팀이 함께 수립해야...,1.000000,1.000000,0.18
2,인사 변화 공지는 언제 이루어지나요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n| 단계 | 설명 |...,인사 변화 공지는 2025년 9월 30일에 이루어집니다.,인사 변화 공지는 2025-09-30에 인사팀에 의해 이루어집니다.,1.000000,1.000000,0.67
3,인사 변화 후 어떤 점을 모니터링해야 하나요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n| 단계 | 설명 |...,인사 변화 후 모니터링해야 할 주요 사항은 다음과 같습니다.\n\n1. 인사 변화가...,인사 변화 후에는 조직 및 팀 성과에 미치는 영향을 지속적으로 모니터링해야 합니다.,1.000000,0.944444,0.14
4,조직 구조 개편안의 목적은 무엇인가요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일:...,조직 구조 개편안의 목적은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 ...,조직 구조 개편은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 위한 전략...,1.000000,1.000000,1.00
5,조직 구조 개편안의 적용 범위는 어떻게 되나요?,[검색 결과:\n-----\n# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일:...,"조직 구조 개편안의 적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)이며, 대상...","적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)입니다.",1.000000,1.000000,0.67
6,조직 구조 개편안의 실행 계획에서 1단계는 무엇인가요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n\n| 단계 | 내용...,"조직 구조 개편안의 실행 계획 1단계는 ""현 조직 구조 분석 및 문제점 도출""입니다...","1단계는 현 조직 구조 분석 및 문제점 도출로, 기간은 2주이며 책임자는 인사팀입니다.",1.000000,1.000000,1.00
7,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 무엇인가요?,[검색 결과:\n-----\n## 4. 리스크 및 대응 방안\n\n| 리스크 | 설...,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 다음과 같습니다.\n\n- 충분...,직원 저항에 대한 대응 방안은 충분한 커뮤니케이션 및 교육 제공입니다.,1.000000,0.375000,0.22
8,핵심 인재 유출 리스크를 평가하기 위한 첫 번째 단계는 무엇인가요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n\n### 단계 1:...,"핵심 인재 유출 리스크 평가의 첫 번째 단계는 ""리스크 진단""입니다. 이 단계에서는...","첫 번째 단계는 리스크 진단이며, 목표는 현재 인재 유출 리스크를 평가하는 것입니다.",1.000000,1.000000,0.80
9,인재 유출을 예방하기 위한 조치에는 어떤 것들이 있나요?,[검색 결과:\n-----\n## 3. 실행 계획 (단계별)\n\n### 단계 1:...,인재 유출을 예방하기 위한 조치는 다음과 같습니다.\n\n1. 보상 체계 재검토 및...,"인재 유출을 예방하기 위한 조치로는 보상 체계 재검토 및 개선, 경력 개발 및 교육...",1.000000,0.555556,0.57


### 평가지표 해석 및 개선방안

1. context_recall (문맥 리콜, 0~1):

    검색된 문서나 문맥에 답변 작성을 위한 필요한 정보가 얼마나 포함되어 있는지를 나타냅니다. 값이 0.5라면, 필요한 정보가 절반 정도는 검색되어 있다는 뜻으로, 문맥의 충분성이 보통 수준임을 뜻합니다.

2. faithfulness (충실도, 0~1):

    생성된 답변이 검색된 문맥 내 정보에 얼마나 사실적으로 충실한지를 평가합니다. 0.5 정도면 절반 정도 문맥 정보를 충실히 반영했으나, 환각(허위 정보) 발생 가능성이 중간 정도임을 의미합니다.

3. factual_correctness (사실 정확성, F1 점수, 0~1)

    생성 답변의 사실적 정확도를 정밀도와 재현율의 조합으로 평가하는 지표입니다. 0.17은 비교적 낮은 점수로, 생성 답변에 사실과 다른 정보가 많이 포함되었을 가능성이 높음을 뜻합니다. 즉, 환각 현상이 꽤 존재할 수 있습니다.


**개선 방안**

- **검색 성능 향상**: 더 정확한 임베딩 모델이나 하이브리드 검색 방법을 고려한다
- **프롬프트 최적화**: 더 구체적이고 명확한 지시사항을 포함한 프롬프트를 설계한다
- **데이터 품질 개선**: 더 정확하고 완전한 참조 데이터를 확보한다
- **후처리 단계 추가**: 사실 검증 단계를 추가하여 정확성을 향상시킨다
- **다양한 검색 전략**(하이브리드 검색, 리랭킹 등) 실험
- **커스텀 평가 메트릭 개발**
- **실시간 평가 시스템 구축**
- **A/B 테스트를 통한 시스템 최적화**